In [42]:
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Flatten,Conv2D, Dropout, LeakyReLU, Reshape, Conv2DTranspose
from keras.datasets.mnist import load_data
from numpy import ones
from numpy.random import randint
from numpy.random import rand, randn
from numpy import zeros

In [43]:
(train_dados, train_rotulos),(teste_dados, teste_rotulos) = load_data()
train_dados = train_dados/255.0
teste_dados = teste_dados/255.0

In [44]:
def descriminador(input_shape = (28,28,1)):
  model = Sequential()
  model.add(Conv2D(64,(3,3), strides=(2,2), padding = 'same', input_shape = input_shape))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.4))
  model.add(Conv2D(64,(3,3), strides=(2,2), padding = 'same'))
  model.add(Dropout(0.4))
  model.add(Flatten())
  model.add(Dense(1,activation='sigmoid'))
  opt =Adam(lr = 0.0002, beta_1= 0.5)
  model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics= ['accuracy'])
  return model

In [45]:
def geradora(dim_latente):
  modelo = Sequential()
  num_neuronios = 128*7*7
  modelo.add(Dense(num_neuronios, input_shape = dim_latente))
  modelo.add(LeakyReLU(alpha = 0.2))
  modelo.add(Reshape((7,7,128)))
  modelo.add(Conv2DTranspose(128,(4,4),strides = (2,2), padding = 'same'))
  modelo.add(LeakyReLU(alpha = 0.2))
  modelo.add(Conv2DTranspose(128,(4,4),strides = (2,2), padding = 'same'))
  modelo.add(LeakyReLU(alpha = 0.2))
  modelo.add(Conv2D(1,(7,7), activation = 'sigmoid', padding = 'same'))
  return modelo

In [46]:
def gan(modelo_g, modelo_d):
  modelo_d.trainable = False
  modelo = Sequential()
  modelo.add(modelo_g)
  modelo.add(modelo_d)
  opt = Adam(lr = 0.0002, beta_1= 0.5)
  modelo.compile(loss = 'binary_crossentropy', optimizer = opt)
  return modelo

In [47]:
def gera_pontos_latentes(dim_latente, num_amostras):
	dados = randn(dim_latente * num_amostras)
	dados = dados.reshape(num_amostras, dim_latente)
	return dados

In [48]:
def gera_reais(dados, num_amostras):
	indices = randint(0, dados.shape[0], num_amostras)
	dados_reais = dados[indices]
	rotulos_reais = ones((num_amostras, 1))
	return dados_reais, rotulos_reais

In [49]:
def gera_falsos(num_amostras):
    dados_falsos = rand(28 * 28 * num_amostras)
    dados_falsos = dados_falsos.reshape(num_amostras, 28, 28, 1)
    rotulos_falsos = zeros((num_amostras, 1))
    return dados_falsos, rotulos_falsos

In [50]:
def gera_falsos_redegeradora(modelo_geradora, num_amostras, dim_latente):
    pontos_latentes = gera_pontos_latentes(dim_latente, num_amostras)
    dados_falsos_gerados = modelo_geradora.predict(pontos_latentes)
    rotulos_falsos = zeros((num_amostras, 1))
    return dados_falsos_gerados, rotulos_falsos

In [51]:
def treina(modelo_g, modelo_d, modelo_gan, dados, dim_latente, num_epocas=100, tam_batch=256):
    batches_por_epoca = int(dados.shape[0] / tam_batch)
    metade_batch = tam_batch // 2
    for i in range(num_epocas):
        for j in range(batches_por_epoca):
            dados_reais, rotulos_reais = gera_reais(dados, metade_batch)
            modelo_d.train_on_batch(dados_reais, rotulos_reais)
            dados_falsos, rotulos_falsos = gera_falsos(metade_batch)
            modelo_d.train_on_batch(dados_falsos, rotulos_falsos)
            dados_gan = gera_pontos_latentes(dim_latente, tam_batch)
            rotulos_gan = ones((tam_batch, 1))
            gan_perda = modelo_gan.train_on_batch(dados_gan, rotulos_gan)